# Season 20 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [1]:
import pandas
%matplotlib inline
from blaseball_mike.models import *
from blaseball_mike.tables import StatType
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot
from IPython.display import display, Markdown
from copy import deepcopy

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

# Fix Attractors
real_pies = deepcopy(pies)
for p in real_pies.lineup + real_pies.rotation:
    p.batting_rating = None
    p.pitching_rating = None
    p.baserunning_rating = None
    p.defense_rating = None

In [2]:
sim = SimulationData.load()
display(Markdown(f"**Last Updated Season {sim.season}, Day {sim.day}**"))

**Last Updated Season 20, Day 113**

---
## Decrees

Decrees are chosen by majority votes across all teams. `1` decrees will pass this season.

### Sun 3
**Set Sun. If your Team sweeps all 3 games of a Regular Season series, the final non-loss is worth 2 Wins.**

See below for the number of additional Wins this would have added. It is unknown how this interacts with the current
Turntables/SunSun mess.

In [3]:
extra_wins = 0
s19_games = Game.load_by_season(season=19, team_id=pies.id)
for i in range(0, len(s19_games), 3):
    series = list(s19_games.values())[i:i+3]
    if series[0].winning_team.id == pies.id and series[1].winning_team.id == pies.id and series[1].winning_team.id == pies.id:
        extra_wins += 1
print(f"Pies would have an extra {extra_wins} Wins")

Pies would have an extra 16 Wins


### Sun 9

**Set Sun. Non-losses on the Week End (every 9th Day) are worth 2 Wins.**

See below for the number of additional Wins this would have added. It is unknown how this interacts with the current
Turntables/SunSun mess.

In [4]:
extra_wins = 0
for g in s19_games.values():
    if (g.day - 1) % 9 == 8 and g.day <= 99:
        if g.winning_team.id == pies.id:
            extra_wins += 1
print(f"Pies would have an extra {extra_wins} Wins")

Pies would have an extra 11 Wins


### Sun 30

**Set Sun. When a game enters Extra Innings, both Teams will immediately earn 1 Win.**

See below for the number of additional Wins this would have added. It is unknown how this interacts with the current
Turntables/SunSun mess.

In [5]:
extra_wins = 0
for g in s19_games.values():
    if g.inning > 9:
        extra_wins += 1
print(f"Pies would have an extra {extra_wins} Wins")

Pies would have an extra 4 Wins


___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills. The bottom 4 teams in the league by
standings will draw `3`. The chosen wills are random, but wills with more votes are more likely to be selected.

### Equivalent Exchange
**Swap a Player from a Team in the League with an Equivalent (within 2 Stars) Player on your Team.**

See below for the 20 best swap candidates for a particular player, sorted by highest stats in the applicable position.

In [6]:
league = League.load()
all_teams = list(league.teams.values())
all_active = []
for team in all_teams:
    if team.id != pies.id:
        all_active.extend(team.lineup + team.rotation)
all_total_stars = get_total_stars(all_active)

In [7]:
for player in pies.lineup:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_active if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.batting_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_batting_stlats([player] + player_matches[0:20]), maxVal=1.5))

####Eduardo Woodman

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Eduardo Woodman,Pies,14.3,5.6,1.8,3.7,3.3
Howell Franklin,Sunbeams,13.0,6.1,1.5,2.6,2.7
Jacob Haynes,Flowers,13.0,5.6,2.9,2.5,1.9
Baby Triumphant,Firefighters,16.0,5.6,3.0,3.4,4.0
Agan Harrison,Shoe Thieves,13.5,5.5,2.0,3.7,2.3
Alston Cerveza,Crabs,14.2,5.5,3.0,2.6,3.1
Dudley Mueller,Spies,16.2,5.4,3.1,3.3,4.5
Fish Summer,Tacos,12.8,5.2,1.8,3.6,2.3
Gia Holbrook,Mechanics,14.0,5.1,3.8,2.4,2.8


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.126442,1.345315,1.152985,0.970888,0.876160,0.283687,0.749895,0.001000,1.025183
Howell Franklin,1.226470,1.107821,1.353333,1.712056,1.274572,0.001000,1.217685,0.001000,1.402964
Jacob Haynes,1.126749,1.334891,1.020797,1.141949,1.265377,0.001000,0.440711,0.013502,0.883104
Baby Triumphant,1.114274,1.142097,1.325945,0.831815,0.924658,0.001000,0.421083,0.001000,1.047283
Agan Harrison,1.107166,1.279644,1.262325,0.913906,0.836783,0.517896,0.676464,0.149649,1.215009
Alston Cerveza,1.103390,1.265764,0.964120,1.163440,1.251674,0.065767,1.319157,0.151105,0.611425
Dudley Mueller,1.079778,1.644472,0.776783,0.987971,1.021591,0.182529,1.023215,0.001000,1.719870
Fish Summer,1.036398,1.053025,1.122064,0.876586,0.903681,0.001000,0.774672,0.001000,1.224700
Gia Holbrook,1.013860,1.396818,0.752736,1.133225,0.902763,0.054784,0.875004,0.001000,1.251697
Basilio Mason,1.013119,1.096140,0.856253,1.150045,1.556136,0.218692,1.216567,0.001000,1.543329


####Beasley Day

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Beasley Day,Pies,17.8,5.4,4.1,4.4,3.9
Thomas Dracaena,Millennials,19.1,6.3,3.4,4.8,4.7
Nagomi Nava,Sunbeams,17.5,6.1,3.0,4.9,3.5
Vessalius Sundae,Worms,17.5,5.9,3.7,4.3,3.7
Gerund Pantheocide,Lift,17.8,5.7,4.1,3.4,4.6
Margarito Nava,Flowers,17.5,5.6,4.1,4.7,3.0
Cannonball Sports,Mechanics,16.7,5.6,2.1,4.4,4.6
Baby Triumphant,Firefighters,16.0,5.6,3.0,3.4,4.0
Baby Doyle,Jazz Hands,17.0,5.5,2.9,3.7,4.8


...

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Peanut Holloway,Pies,12.1,4.5,1.5,2.4,3.7
Howell Franklin,Sunbeams,13.0,6.1,1.5,2.6,2.7
Jacob Haynes,Flowers,13.0,5.6,2.9,2.5,1.9
Agan Harrison,Shoe Thieves,13.5,5.5,2.0,3.7,2.3
Fish Summer,Tacos,12.8,5.2,1.8,3.6,2.3
Summers Preston,Dale,11.0,5.1,1.9,1.9,2.1
Gia Holbrook,Mechanics,14.0,5.1,3.8,2.4,2.8
Rat Mason,Tacos,10.2,5.0,1.3,2.4,1.5
Comfort Septemberish,Spies,13.7,4.9,1.2,4.9,2.6


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Peanut Holloway,0.902187,0.813171,1.014025,1.047254,0.996649,0.001000,0.272813,0.716477,1.048080
Howell Franklin,1.226470,1.107821,1.353333,1.712056,1.274572,0.001000,1.217685,0.001000,1.402964
Jacob Haynes,1.126749,1.334891,1.020797,1.141949,1.265377,0.001000,0.440711,0.013502,0.883104
Agan Harrison,1.107166,1.279644,1.262325,0.913906,0.836783,0.517896,0.676464,0.149649,1.215009
Fish Summer,1.036398,1.053025,1.122064,0.876586,0.903681,0.001000,0.774672,0.001000,1.224700
Summers Preston,1.015975,0.945562,1.285928,1.267384,0.403316,0.001000,0.896441,0.001000,0.816163
Gia Holbrook,1.013860,1.396818,0.752736,1.133225,0.902763,0.054784,0.875004,0.001000,1.251697
Rat Mason,1.009065,1.290628,0.798484,1.077502,0.906716,0.030115,1.092972,0.001000,0.653243
Comfort Septemberish,0.978574,1.177996,1.187902,0.375357,0.767285,0.611622,1.105479,0.060000,0.832023
Lotus Mango,0.953246,0.943710,1.181527,0.593847,0.761216,0.067847,0.317959,0.001000,0.642309


####Usurper Violet

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Usurper Violet,Pies,11.7,2.9,3.2,1.7,3.9
Howell Franklin,Sunbeams,13.0,6.1,1.5,2.6,2.7
Jacob Haynes,Flowers,13.0,5.6,2.9,2.5,1.9
Agan Harrison,Shoe Thieves,13.5,5.5,2.0,3.7,2.3
Fish Summer,Tacos,12.8,5.2,1.8,3.6,2.3
Summers Preston,Dale,11.0,5.1,1.9,1.9,2.1
Rat Mason,Tacos,10.2,5.0,1.3,2.4,1.5
Scores Baserunner,Flowers,12.9,4.7,1.2,2.9,4.0
Denzel Scott,Spies,12.9,4.5,3.1,2.6,2.7


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176
Howell Franklin,1.226470,1.107821,1.353333,1.712056,1.274572,0.001000,1.217685,0.001000,1.402964
Jacob Haynes,1.126749,1.334891,1.020797,1.141949,1.265377,0.001000,0.440711,0.013502,0.883104
Agan Harrison,1.107166,1.279644,1.262325,0.913906,0.836783,0.517896,0.676464,0.149649,1.215009
Fish Summer,1.036398,1.053025,1.122064,0.876586,0.903681,0.001000,0.774672,0.001000,1.224700
Summers Preston,1.015975,0.945562,1.285928,1.267384,0.403316,0.001000,0.896441,0.001000,0.816163
Rat Mason,1.009065,1.290628,0.798484,1.077502,0.906716,0.030115,1.092972,0.001000,0.653243
Scores Baserunner,0.940087,1.158595,0.974880,0.631934,0.521151,0.077682,0.458698,0.136667,0.380176
Denzel Scott,0.906225,1.005499,0.925789,0.609571,0.704572,0.076566,0.740125,0.001000,0.998163
Math Velazquez,0.890787,0.952254,0.999542,0.521371,0.991741,0.453741,0.440042,0.201246,0.494748


In [8]:
for player in pies.rotation:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_active if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.pitching_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_pitching_stlats([player] + player_matches[0:20]), maxVal=1.5))

####Tiana Takahashi

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Tiana Takahashi,Pies,16.6,2.7,5.3,4.0,4.6
Montgomery Bullock,Crabs,14.7,1.6,6.6,2.4,4.0
Sigmund Castillo,Sunbeams,16.2,3.3,6.4,4.8,1.6
Cell Barajas,Wild Wings,17.1,3.9,6.2,3.3,3.6
Juice Collins,Fridays,16.5,2.8,6.2,4.1,3.4
Howell Rocha,Dale,18.1,4.1,6.0,4.0,3.9
Silvia Rugrat,Wild Wings,15.3,3.7,6.0,2.4,3.1
Sam Scandal,Steaks,17.8,4.9,5.9,4.2,2.8
Mindy Kugel,Mechanics,18.3,5.0,5.7,4.0,3.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Tiana Takahashi,1.064998,1.142441,1.215111,1.182198,0.356126,0.872711,1.199716
Montgomery Bullock,1.318505,1.738706,0.999277,1.152921,0.742616,1.412281,0.908350
Sigmund Castillo,1.289484,1.302577,1.068161,1.400424,1.429219,1.460265,1.094387
Cell Barajas,1.249351,1.247970,1.311778,1.116352,1.047708,0.489513,0.467608
Juice Collins,1.245491,1.279794,1.257108,0.943706,1.266897,0.662010,1.331051
Howell Rocha,1.205012,1.260072,1.167638,1.028929,1.115917,0.776210,0.738377
Silvia Rugrat,1.198656,1.189104,1.383893,0.799356,0.908543,1.367479,1.468683
Sam Scandal,1.178393,1.229786,1.193732,0.858288,1.179120,0.863933,1.022701
Mindy Kugel,1.144480,1.062422,1.247913,1.023672,1.075738,1.234935,0.461299
Patty Fox,1.141465,1.306372,1.210541,0.678718,0.935361,0.596270,0.747740


####Greer Lott

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Greer Lott,Pies,19.0,3.9,6.2,3.9,5.0
Burke Gonzales,Wild Wings,20.5,4.4,6.9,4.6,4.6
Curry Aliciakeyes,Magic,20.2,4.5,6.4,3.5,5.9
Cell Barajas,Wild Wings,17.1,3.9,6.2,3.3,3.6
Howell Rocha,Dale,18.1,4.1,6.0,4.0,3.9
Sam Scandal,Steaks,17.8,4.9,5.9,4.2,2.8
Mindy Kugel,Mechanics,18.3,5.0,5.7,4.0,3.5
Leach Ingram,Breath Mints,19.5,4.3,5.6,4.6,4.9
Yummy Elliott,Tacos,17.8,4.0,5.6,3.9,4.3


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Greer Lott,1.233730,1.619256,1.082263,1.033680,0.497049,1.099649,0.471501
Burke Gonzales,1.371320,1.342423,1.499646,1.028373,0.988620,1.145311,0.948175
Curry Aliciakeyes,1.275548,1.164682,1.309874,1.367723,1.061156,1.285113,0.991654
Cell Barajas,1.249351,1.247970,1.311778,1.116352,1.047708,0.489513,0.467608
Howell Rocha,1.205012,1.260072,1.167638,1.028929,1.115917,0.776210,0.738377
Sam Scandal,1.178393,1.229786,1.193732,0.858288,1.179120,0.863933,1.022701
Mindy Kugel,1.144480,1.062422,1.247913,1.023672,1.075738,1.234935,0.461299
Leach Ingram,1.124552,1.310929,0.752549,1.519332,1.225849,1.657216,1.172738
Yummy Elliott,1.112591,0.963827,1.310590,1.291143,0.788595,1.098279,1.079743
Kennedy Loser,1.104362,1.191634,1.109731,0.607264,1.506699,1.163425,1.321608


####Henry Marshallow

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Henry Marshallow,Pies,15.5,3.8,4.1,2.9,4.8
Montgomery Bullock,Crabs,14.7,1.6,6.6,2.4,4.0
Sigmund Castillo,Sunbeams,16.2,3.3,6.4,4.8,1.6
Cell Barajas,Wild Wings,17.1,3.9,6.2,3.3,3.6
Juice Collins,Fridays,16.5,2.8,6.2,4.1,3.4
Silvia Rugrat,Wild Wings,15.3,3.7,6.0,2.4,3.1
Patty Fox,Millennials,15.1,4.6,5.7,1.6,3.2
Mindy Salad,Lovers,16.4,3.0,5.6,2.9,4.9
Sandford Garner,Lovers,17.0,4.4,5.5,3.7,3.4


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.812533,0.523573,1.209625,1.470785,0.867855,0.856246,0.905731
Montgomery Bullock,1.318505,1.738706,0.999277,1.152921,0.742616,1.412281,0.908350
Sigmund Castillo,1.289484,1.302577,1.068161,1.400424,1.429219,1.460265,1.094387
Cell Barajas,1.249351,1.247970,1.311778,1.116352,1.047708,0.489513,0.467608
Juice Collins,1.245491,1.279794,1.257108,0.943706,1.266897,0.662010,1.331051
Silvia Rugrat,1.198656,1.189104,1.383893,0.799356,0.908543,1.367479,1.468683
Patty Fox,1.141465,1.306372,1.210541,0.678718,0.935361,0.596270,0.747740
Mindy Salad,1.113563,0.995304,1.348504,1.090404,0.826068,0.867468,0.468365
Sandford Garner,1.097436,1.133224,0.860195,1.620295,1.219319,0.940324,1.437379
Inky Rutledge,1.092614,1.515457,0.864538,0.624700,1.113789,0.950643,1.082086


### Swap
**Swap two Players on your Team's Roster.**

This allows switching the position of any two players, including from the shadows. See below for swapping various
players into different positions.
**Batting**

In [9]:
best_batting_shadow = sort_shadows(pies, stat="hitting", num=1, order="best")[0]
best_pitching_shadow = sort_shadows(pies, stat="pitching", num=1, order="best")[0]
bhp = best_hitting_pitcher(real_pies)
bph = best_pitching_hitter(real_pies)
worst_batter = sort_lineup(real_pies, num=1)[0]
worst_pitcher = sort_rotation(real_pies, num=1)[0]

display(Markdown(f"Swapping {worst_batter.name} and {best_batting_shadow.name}"))
display(pandas.DataFrame(swap_players(pies, worst_batter, best_batting_shadow)))

display(Markdown(f"Swapping {worst_batter.name} and {bhp.name}"))
display(pandas.DataFrame(swap_players(pies, worst_batter, bhp)))

Swapping Usurper Violet and Doc Anice

,Pies
batting_change,+3.6790%
pitching_change,+0.0000%
baserunning_change,+3.8868%
defense_change,+1.0285%


Swapping Usurper Violet and Greer Lott

,Pies
batting_change,+2.4933%
pitching_change,-19.9398%
baserunning_change,+5.6660%
defense_change,+2.6620%


**Pitching**

In [10]:
display(Markdown(f"Swapping {worst_pitcher.name} and {best_pitching_shadow.name}"))
display(pandas.DataFrame(swap_players(pies, worst_pitcher, best_pitching_shadow)))

display(Markdown(f"Swapping {worst_pitcher.name} and {bph.name}"))
display(pandas.DataFrame(swap_players(pies, worst_pitcher, bph)))

Swapping Henry Marshallow and Lucy Tokkan

,Pies
batting_change,+0.0000%
pitching_change,-1.1110%
baserunning_change,+0.0000%
defense_change,+0.0000%


Swapping Henry Marshallow and Jaxon Buckley

,Pies
batting_change,-8.3718%
pitching_change,+2.2523%
baserunning_change,-2.8354%
defense_change,+2.3478%


### Move
**Move a Player on your Team to the desired location on your Roster.**

Move would move a player from any location on the team (including Shadows) to another position, increasing the number
of players in that position. See below for moving various players into different positions.

**Batting**

In [11]:
display(Markdown(f"Moving {worst_batter.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_batter, "shadows")))

display(Markdown(f"Moving {best_batting_shadow.name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, best_batting_shadow, "lineup")))

display(Markdown(f"Moving {bhp.name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, bhp, "lineup")))

Moving Usurper Violet to the Shadows

,Pies
batting_change,+5.5109%
pitching_change,+0.0000%
baserunning_change,+5.7842%
defense_change,-0.1157%


Moving Doc Anice to the Lineup

,Pies
batting_change,-1.0160%
pitching_change,+0.0000%
baserunning_change,-1.0439%
defense_change,+1.0042%


Moving Greer Lott to the Lineup

,Pies
batting_change,-2.0700%
pitching_change,-9.8321%
baserunning_change,+0.5376%
defense_change,+2.4562%


**Pitching**

In [12]:
display(Markdown(f"Moving {worst_pitcher.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_pitcher, "shadows")))

display(Markdown(f"Moving {best_pitching_shadow.name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, best_pitching_shadow, "rotation")))

display(Markdown(f"Moving {bph.name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, bph, "rotation")))

Moving Henry Marshallow to the Shadows

,Pies
batting_change,+0.0000%
pitching_change,+11.2277%
baserunning_change,+0.0000%
defense_change,+0.0000%


Moving Lucy Tokkan to the Rotation

,Pies
batting_change,+0.0000%
pitching_change,-6.4471%
baserunning_change,+0.0000%
defense_change,+0.0000%


Moving Jaxon Buckley to the Rotation

,Pies
batting_change,-6.6040%
pitching_change,-3.9246%
baserunning_change,-0.9578%
defense_change,+0.0438%


### Take
**Take an Item, repair it, and move it to a Player on your Team.**

This would steal an item from another team and give it to a chosen player on our team. The player receiving the item
will drop a random item if there are no open item slots. See below for a list of all non-base items held by players in
the league.

In [13]:
players = Player.load_all()
carried_items = []
for p in players.values():
    team = getattr(p,"_league_team_id", None)
    if team == pies.id or team is None:
        continue
    carried_items.extend([x for x in p.items if len(x.adjustments) > 1])
carried_items.sort(key=lambda x: len(x.adjustments), reverse=True)
get_item_stats(carried_items, include_wielder=True)

,Max Durability,Modifications
Slimy Frosty Cap of Wisdom,4,[]
Frosty Hot Air Ring,4,[]
Sharp Gravity Holey Glove,4,[Gravity]
Lucky Travelling Shoes of Wisdom,4,[Traveling]
Greedy Noise-Cancelling Air Socks,4,[Soundproof]
Slimy Hot Skate Helmet,4,[Pro Skater]
Holey Rubber Sunglasses,3,[]
Greedy Metaphorical Bat,3,[]
Underhanded Cool Greedy Sunglasses,4,[Underhanded]
Weird Plant-Based Cap,3,[]


### Shadow Infuse
**Boost a random Stat Category of a Player in your Shadows by 20% to 40%.**

See below for how this would affect various team members at the 2 extreme percentages. This will only affect one player.

#### Batting

In [14]:
display(Markdown("**20% Change**"))
infuse_batting_table, _, _ = improve_team_batting_table(real_pies, 0.20, shadows=True)
display(infuse_batting_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.20, shadows=True)), maxVal=1))

**20% Change**

,old_batting_stars,new_batting_stars,change_in_batting_stars
Adrian Melon,2.353291,3.451552,1.098261
August Obrien,3.408347,4.488341,1.079995
Mcfarland Vargas,1.383943,2.686492,1.302549
Twofurious Puddles,2.635533,3.759716,1.124183
Mummy Cantburn,1.390958,2.681958,1.291000
Doginic Burgertoes,2.079285,3.353370,1.274085
Nerd Pacheco,0.270478,1.359404,1.088926
Yusef Puddles,2.390547,3.545751,1.155204
Sheev Shriffle,3.352957,4.372160,1.019204
Adeline Hammer,1.095992,2.454162,1.358170


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Adrian Melon,0.690310,0.502171,0.908122,1.151649,0.302259,0.075678,0.532170,0.010000,0.743764
August Obrien,0.897668,0.841217,1.127569,1.113891,0.692708,0.723854,0.756849,0.010000,0.663958
Mcfarland Vargas,0.537298,0.317084,0.976150,0.507714,0.375366,0.784279,0.617832,0.010000,0.751343
Twofurious Puddles,0.751943,0.777625,0.680623,1.057061,1.176273,0.612102,0.209182,0.010000,0.674513
Mummy Cantburn,0.536392,0.295592,0.776609,1.147265,0.725890,0.716564,0.212553,0.010000,1.127160
Doginic Burgertoes,0.670674,0.376724,1.184426,0.305629,1.134521,0.012919,0.238098,0.561237,0.504905
Nerd Pacheco,0.271881,0.241707,0.252607,0.217167,0.232564,0.744911,0.439614,0.779935,0.227951
Yusef Puddles,0.709150,1.268932,0.423433,0.902603,0.391137,0.419418,0.351715,0.010000,1.250129
Sheev Shriffle,0.874432,1.184880,0.810356,0.421637,0.682530,0.402199,0.974400,0.029204,0.753719
Adeline Hammer,0.490832,0.244688,0.606439,0.939264,1.174677,0.526302,0.583623,0.268421,0.483524


In [15]:
display(Markdown("**40% Change**"))
infuse_batting_table, _, _ = improve_team_batting_table(real_pies, 0.40, shadows=True)
display(infuse_batting_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.40, shadows=True)), maxVal=1))

**40% Change**

,old_batting_stars,new_batting_stars,change_in_batting_stars
Adrian Melon,2.353291,4.418199,2.064908
August Obrien,3.408347,5.455504,2.047157
Mcfarland Vargas,1.383943,3.743373,2.359429
Twofurious Puddles,2.635533,4.728474,2.092941
Mummy Cantburn,1.390958,3.735507,2.344550
Doginic Burgertoes,2.079285,4.391437,2.312151
Nerd Pacheco,0.270478,2.364572,2.094095
Yusef Puddles,2.390547,4.582121,2.191574
Sheev Shriffle,3.352957,5.332400,1.979444
Adeline Hammer,1.095992,3.521267,2.425275


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Adrian Melon,0.883640,0.702171,1.108122,1.351649,0.502259,0.010000,0.732170,0.010000,0.943764
August Obrien,1.091101,1.041217,1.327569,1.313891,0.892708,0.523854,0.956849,0.010000,0.863958
Mcfarland Vargas,0.748675,0.517084,1.176150,0.707714,0.575366,0.584279,0.817832,0.010000,0.951343
Twofurious Puddles,0.945695,0.977625,0.880623,1.257061,1.376273,0.412102,0.409182,0.010000,0.874513
Mummy Cantburn,0.747101,0.495592,0.976609,1.347265,0.925890,0.516564,0.412553,0.010000,1.327160
Doginic Burgertoes,0.878287,0.576724,1.384426,0.505629,1.334521,0.010000,0.438098,0.361237,0.704905
Nerd Pacheco,0.472914,0.441707,0.452607,0.417167,0.432564,0.544911,0.639614,0.579935,0.427951
Yusef Puddles,0.916424,1.468932,0.623433,1.102603,0.591137,0.219418,0.551715,0.010000,1.450129
Sheev Shriffle,1.066480,1.384880,1.010356,0.621637,0.882530,0.202199,1.174400,0.010000,0.953719
Adeline Hammer,0.704253,0.444688,0.806439,1.139264,1.374677,0.326302,0.783623,0.068421,0.683524


#### Pitching

In [16]:
display(Markdown("**20% Change**"))
infuse_pitching_table, _, _ = improve_team_pitching_table(real_pies, 0.20, shadows=True)
display(infuse_pitching_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.20, shadows=True)), maxVal=1))

**20% Change**

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Adrian Melon,0.939306,2.326798,1.387492
August Obrien,1.045544,2.111234,1.065690
Mcfarland Vargas,1.501794,2.676728,1.174935
Twofurious Puddles,3.474609,4.620631,1.146021
Mummy Cantburn,2.326166,3.450122,1.123956
Doginic Burgertoes,1.808231,2.965528,1.157297
Nerd Pacheco,0.334608,1.236132,0.901524
Yusef Puddles,2.508852,3.633514,1.124663
Sheev Shriffle,1.942316,3.092375,1.150060
Adeline Hammer,0.915392,2.391269,1.475877


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Adrian Melon,0.465360,0.392291,1.156712,0.221132,0.292483,0.786090,0.929182
August Obrien,0.422247,0.436723,0.461477,1.087830,0.297743,0.300188,0.522708
Mcfarland Vargas,0.535346,0.387950,1.042297,0.486164,0.548503,1.010603,0.549185
Twofurious Puddles,0.924126,1.011110,0.850034,1.007712,0.918615,0.616240,0.925894
Mummy Cantburn,0.690024,0.823505,0.792185,0.439305,0.637026,0.612072,1.172225
Doginic Burgertoes,0.593106,0.789539,0.414470,1.113112,0.511613,0.961430,0.220448
Nerd Pacheco,0.247226,0.319127,0.325662,0.256284,0.256083,0.226155,0.241882
Yusef Puddles,0.726703,0.769080,0.677310,1.233877,0.525880,1.025805,0.817095
Sheev Shriffle,0.618475,0.463763,1.009104,0.679938,0.644447,1.077531,0.841488
Adeline Hammer,0.478254,0.971047,0.241792,0.737204,0.338224,0.964063,1.009582


In [17]:
display(Markdown("**40% Change**"))
infuse_pitching_table, _, _ = improve_team_pitching_table(real_pies, 0.40, shadows=True)
display(infuse_pitching_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.40, shadows=True)), maxVal=1))

**40% Change**

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Adrian Melon,0.939306,3.556168,2.616863
August Obrien,1.045544,3.219652,2.174108
Mcfarland Vargas,1.501794,3.857349,2.355555
Twofurious Puddles,3.474609,5.807659,2.333050
Mummy Cantburn,2.326166,4.608101,2.281935
Doginic Burgertoes,1.808231,4.136959,2.328728
Nerd Pacheco,0.334608,2.240878,1.906270
Yusef Puddles,2.508852,4.799377,2.290526
Sheev Shriffle,1.942316,4.265267,2.322952
Adeline Hammer,0.915392,3.646409,2.731017


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Adrian Melon,0.711234,0.592291,1.356712,0.421132,0.492483,0.986090,1.129182
August Obrien,0.643930,0.636723,0.661477,1.287830,0.497743,0.500188,0.722708
Mcfarland Vargas,0.771470,0.587950,1.242297,0.686164,0.748503,1.210603,0.749185
Twofurious Puddles,1.161532,1.211110,1.050034,1.207712,1.118615,0.816240,1.125894
Mummy Cantburn,0.921620,1.023505,0.992185,0.639305,0.837026,0.812072,1.372225
Doginic Burgertoes,0.827392,0.989539,0.614470,1.313112,0.711613,1.161430,0.420448
Nerd Pacheco,0.448176,0.519127,0.525662,0.456284,0.456083,0.426155,0.441882
Yusef Puddles,0.959875,0.969080,0.877310,1.433877,0.725880,1.225805,1.017095
Sheev Shriffle,0.853053,0.663763,1.209104,0.879938,0.844447,1.277531,1.041488
Adeline Hammer,0.729282,1.171047,0.441792,0.937204,0.538224,1.164063,1.209582


#### Baserunning

In [18]:
display(Markdown("**20% Change**"))
infuse_baserunning_table, _, _ = improve_team_baserunning_table(real_pies, 0.20, shadows=True)
display(infuse_baserunning_table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.20, shadows=True)), maxVal=1))

**20% Change**

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Adrian Melon,3.116697,4.145628,1.028930
August Obrien,3.318856,4.398980,1.080124
Mcfarland Vargas,3.783245,4.734596,0.951351
Twofurious Puddles,1.817993,2.913054,1.095061
Mummy Cantburn,2.021513,3.133820,1.112307
Doginic Burgertoes,1.779564,3.023082,1.243518
Nerd Pacheco,0.553414,1.681272,1.127858
Yusef Puddles,2.938345,3.927098,0.988753
Sheev Shriffle,2.215166,3.254024,1.038858
Adeline Hammer,2.451014,3.433736,0.982722


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Adrian Melon,0.829126,1.040831,0.784640,0.685051,0.376399,0.621241
August Obrien,0.879796,1.166470,0.340389,0.901812,0.681149,0.615344
Mcfarland Vargas,0.946919,1.122765,0.757085,1.012168,0.682377,0.621246
Twofurious Puddles,0.582611,0.393390,0.578033,0.714860,1.088752,1.063079
Mummy Cantburn,0.626764,0.488250,1.032320,1.139393,0.298762,0.959435
Doginic Burgertoes,0.604616,0.522288,0.227095,0.817854,0.827448,1.093013
Nerd Pacheco,0.336254,0.322806,0.225072,0.244925,0.387564,0.246761
Yusef Puddles,0.785420,0.831219,0.449357,1.233696,0.696257,0.583273
Sheev Shriffle,0.650805,0.473349,0.818521,1.011066,1.171078,0.591838
Adeline Hammer,0.686747,0.562092,0.718388,0.868718,1.024803,0.650209


In [19]:
display(Markdown("**40% Change**"))
infuse_baserunning_table, _, _ = improve_team_baserunning_table(real_pies, 0.40, shadows=True)
display(infuse_baserunning_table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.40, shadows=True)), maxVal=1))

**40% Change**

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Adrian Melon,3.116697,5.097653,1.980956
August Obrien,3.318856,5.368964,2.050108
Mcfarland Vargas,3.783245,5.651652,1.868407
Twofurious Puddles,1.817993,3.908728,2.090735
Mummy Cantburn,2.021513,4.128766,2.107253
Doginic Burgertoes,1.779564,4.022373,2.242810
Nerd Pacheco,0.553414,2.677492,2.124078
Yusef Puddles,2.938345,4.866021,1.927676
Sheev Shriffle,2.215166,4.224486,2.009320
Adeline Hammer,2.451014,4.374434,1.923420


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Adrian Melon,1.019531,1.240831,0.984640,0.885051,0.576399,0.821241
August Obrien,1.073793,1.366470,0.540389,1.101812,0.881149,0.815344
Mcfarland Vargas,1.130330,1.322765,0.957085,1.212168,0.882377,0.821246
Twofurious Puddles,0.781746,0.593390,0.778033,0.914860,1.288752,1.263079
Mummy Cantburn,0.825753,0.688250,1.232320,1.339393,0.498762,1.159435
Doginic Burgertoes,0.804475,0.722288,0.427095,1.017854,1.027448,1.293013
Nerd Pacheco,0.535498,0.522806,0.425072,0.444925,0.587564,0.446761
Yusef Puddles,0.973204,1.031219,0.649357,1.433696,0.896257,0.783273
Sheev Shriffle,0.844897,0.673349,1.018521,1.211066,1.371078,0.791838
Adeline Hammer,0.874887,0.762092,0.918388,1.068718,1.224803,0.850209


#### Defense

In [20]:
display(Markdown("**20% Change**"))
infuse_defense_table, _, _ = improve_team_defense_table(real_pies, 0.20, shadows=True)
display(infuse_defense_table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.20, shadows=True)), maxVal=1))

**20% Change**

,old_defense_stars,new_defense_stars,change_in_defense_stars
Adrian Melon,2.664412,3.626229,0.961817
August Obrien,2.035102,3.265688,1.230586
Mcfarland Vargas,2.071032,3.315858,1.244825
Twofurious Puddles,3.610790,4.445614,0.834824
Mummy Cantburn,3.289650,4.105055,0.815405
Doginic Burgertoes,1.538638,2.876826,1.338187
Nerd Pacheco,0.500829,1.835336,1.334507
Yusef Puddles,3.712285,4.522409,0.810124
Sheev Shriffle,0.873827,2.631009,1.757183
Adeline Hammer,3.312777,4.176898,0.864121


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Adrian Melon,0.725246,0.388840,0.543662,1.105908,0.716713,1.136550
August Obrien,0.653138,0.254615,0.683368,1.037103,0.527928,0.852261
Mcfarland Vargas,0.663172,0.254401,0.556182,0.790229,0.941915,1.104134
Twofurious Puddles,0.889123,1.047265,1.109861,0.460410,0.855028,0.580553
Mummy Cantburn,0.821011,0.791982,0.596224,0.716128,0.825545,1.055624
Doginic Burgertoes,0.575365,0.247259,1.064391,0.615999,0.318550,0.292532
Nerd Pacheco,0.367067,0.230500,0.252892,0.251010,0.233923,0.222577
Yusef Puddles,0.904482,0.716601,1.202048,0.515498,0.841809,1.138046
Sheev Shriffle,0.526202,0.338111,0.203537,0.458516,0.663167,1.130159
Adeline Hammer,0.835380,0.691199,0.490750,1.197193,1.194874,1.005600


In [21]:
display(Markdown("**40% Change**"))
infuse_defense_table, _, _ = improve_team_defense_table(real_pies, 0.40, shadows=True)
display(infuse_defense_table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.40, shadows=True)), maxVal=1))

**40% Change**

,old_defense_stars,new_defense_stars,change_in_defense_stars
Adrian Melon,2.664412,4.442809,1.778397
August Obrien,2.035102,4.143706,2.108604
Mcfarland Vargas,2.071032,4.198096,2.127063
Twofurious Puddles,3.610790,5.189850,1.579060
Mummy Cantburn,3.289650,4.849029,1.559380
Doginic Burgertoes,1.538638,3.813725,2.275087
Nerd Pacheco,0.500829,2.810050,2.309221
Yusef Puddles,3.712285,5.255879,1.543594
Sheev Shriffle,0.873827,3.582429,2.708603
Adeline Hammer,3.312777,4.942295,1.629518


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Adrian Melon,0.888562,0.588840,0.743662,1.305908,0.916713,1.336550
August Obrien,0.828741,0.454615,0.883368,1.237103,0.727928,1.052261
Mcfarland Vargas,0.839619,0.454401,0.756182,0.990229,1.141915,1.304134
Twofurious Puddles,1.037970,1.247265,1.309861,0.660410,1.055028,0.780553
Mummy Cantburn,0.969806,0.991982,0.796224,0.916128,1.025545,1.255624
Doginic Burgertoes,0.762745,0.447259,1.264391,0.815999,0.518550,0.492532
Nerd Pacheco,0.562010,0.430500,0.452892,0.451010,0.433923,0.422577
Yusef Puddles,1.051176,0.916601,1.402048,0.715498,1.041809,1.338046
Sheev Shriffle,0.716486,0.538111,0.403537,0.658516,0.863167,1.330159
Adeline Hammer,0.988459,0.891199,0.690750,1.397193,1.394874,1.205600


### Shadow Revoke
**Revoke a Player from your Team's Shadows.**

This would remove a player from our shadows by moving them to another team, without providing a replacement.

### Reform
**Re-roll an eligible Permanent Modification for a Player on your Team.**

Below is a list of all players with modifications that could be rerolled. It would be better to reroll modifications
with negative effects.

In [22]:
mod_players = [x for x in pies.lineup + pies.rotation if len(x.perm_attr) > 0]
mod_list = pandas.DataFrame()
for player in mod_players:
    data = pandas.Series({"mods": [x.title for x in player.perm_attr if x.id not in UNREFORMED_MODS]}, name=player.name)
    if len(data["mods"]) == 0:
        continue
    mod_list = mod_list.append(data)
mod_list

,mods
Beasley Day,[Minimalist]
Huber Frumple,[Seeker]
Ruslan Greatness,[Friend of Crows]
Jaxon Buckley,[Fire Eater]
Peanut Holloway,"[Honey Roasted, Negative]"
Greer Lott,[Negative]
Henry Marshallow,[Maximalist]


### Magnify
**Magnify a Player on your Team. Runs batted in or allowed by this Player are multiplied.**

See below for amount of runs batted in/allowed for each player in the previous season. It is unknown what the mutliplier amount is.

In [23]:
table = get_batting_stats(pies.lineup, season=19)
table.filter(items=["runs_batted_in"])

,runs_batted_in
Eduardo Woodman,79.2
Beasley Day,60.0
Huber Frumple,71.6
Ruslan Greatness,74.7
Jaxon Buckley,81.0
Lang Richardson,43.1
Peanut Holloway,44.4
Usurper Violet,36.7


In [24]:
table = get_pitching_stats(pies.rotation, season=19)
table.filter(items=["runs_allowed"])

,runs_allowed
Tiana Takahashi,86.0
Greer Lott,65.0
Henry Marshallow,48.1


___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to one random team, where teams with
more votes are more likely to be selected.

### TWO FREAKIN BATS
**Two random Players in your Team's Lineup with an available Item slot (empty or broken) will open Handcrafted Aluminum Bat Crates.**

[Two of them](https://www.youtube.com/watch?v=btHpHjabRcc). Aluminum bats increase Ground Friction while reducing
Musclitude and Buoyancy. See below for list of batters with an empty item slot.

In [25]:
free_items = has_item_slots(pies.lineup)
get_stars(free_items)

,Combined,Batting,Pitching,Baserunning,Defense
Peanut Holloway,12.1,4.5,1.5,2.4,3.7


### Shadow Capes Supply Run
**All Players in your Team's Shadows with an available Item slot will open a Generic Cape Crate.**

Capes add Watchfulness, which is a defense stat. See below for list of shadow players with open item slots

In [26]:
free_items = has_item_slots(pies.shadows)
get_stars(free_items)

,Combined,Batting,Pitching,Baserunning,Defense
Adrian Melon,9.1,2.4,0.9,3.1,2.7
August Obrien,9.8,3.4,1.0,3.3,2.0
Mcfarland Vargas,8.7,1.4,1.5,3.8,2.1
Twofurious Puddles,11.5,2.6,3.5,1.8,3.6
Mummy Cantburn,9.0,1.4,2.3,2.0,3.3
Doginic Burgertoes,7.2,2.1,1.8,1.8,1.5
Nerd Pacheco,1.7,0.3,0.3,0.6,0.5
Yusef Puddles,11.6,2.4,2.5,2.9,3.7
Sheev Shriffle,8.4,3.4,1.9,2.2,0.9
Adeline Hammer,7.8,1.1,0.9,2.5,3.3


### Shots in the Dark
**Call the Negative Alternates (maintaining Star count) for 5 random Players in your Team's Shadows.**

A Negative Alternate is a player reroll (maintaining Combined Star value), while adding the Negative mod which makes
their eDensity Negative. See below for our current shadows, the target is random.

In [27]:
get_stars(pies.shadows)

,Combined,Batting,Pitching,Baserunning,Defense
Adrian Melon,9.1,2.4,0.9,3.1,2.7
August Obrien,9.8,3.4,1.0,3.3,2.0
Mcfarland Vargas,8.7,1.4,1.5,3.8,2.1
Twofurious Puddles,11.5,2.6,3.5,1.8,3.6
Mummy Cantburn,9.0,1.4,2.3,2.0,3.3
Doginic Burgertoes,7.2,2.1,1.8,1.8,1.5
Nerd Pacheco,1.7,0.3,0.3,0.6,0.5
Yusef Puddles,11.6,2.4,2.5,2.9,3.7
Sheev Shriffle,8.4,3.4,1.9,2.2,0.9
Adeline Hammer,7.8,1.1,0.9,2.5,3.3


### Couldn't Hurt
**Call the Negative Alternates (maintaining Star count) for the 2 worst Players on your Team's Active Roster.**

A Negative Alternate is a player reroll (maintaining Combined Star value), while adding the Negative mod which makes
their eDensity Negative. See below for the 2 worst players on the team.

In [28]:
worst_pies = sort_overall(pies, num=2)
get_stars(worst_pies)

,Combined,Batting,Pitching,Baserunning,Defense
Usurper Violet,11.7,2.9,3.2,1.7,3.9
Lang Richardson,12.3,3.5,1.0,3.8,3.9


### Green Light
**One Player on all Teams in your Division will play 50% better in Polarity Plus Weather and 50% worse in Polarity Minus Weather.**

~Random~. Playing worse in Polarity Minus is good if it hits a batter, because you dont want to score when it would result in unruns.

### Sponge Blob
**Give all Players with Ego on your Team's Roster the Sponge modification.**

See below for list of players with any level of Ego. The effect of the Sponge modification is unknown.

In [29]:
egod = has_mods(pies.lineup + pies.rotation + pies.shadows, ["EGO1", "EGO2", "EGO3", "EGO4"])
get_stars(egod)

,Combined,Batting,Pitching,Baserunning,Defense
Jaxon Buckley,19.4,7.1,4.4,4.0,3.8


### Heavy-Handed & Light-Handed
**Your Team becomes Heavy-Handed, making all Items held by Players on the Team positively eDense.**

**Your Team becomes Light-Handed, making all Items held by Players on the Team negatively eDense.**

Items are currently not contributing to eDensity for either players or teams, so the exact effect of these is unknown.

### Strange Attractor
**Recruit a Random Player in the League with the Attractor Modification to your Team.**

See below for list of all active players with Attractor. This will not remove an existing player from our team.

Notes:
* This lists the False Stars, they will play using their player attributes instead.

In [30]:
attractors = []
for team in all_teams:
    attractors.extend(has_mods(team.lineup + team.rotation, "ATTRACTOR"))
display(get_stars(attractors, include_team=True))

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Liquid Friend,Jazz Hands,21.5,6.8,4.2,5.2,5.3
Alyssa Harrell,Firefighters,22.9,5.8,6.7,4.4,6.0
Chorby Short,Flowers,28.8,6.1,8.7,7.7,6.2
Forrest Best,Steaks,24.1,4.6,7.7,5.7,6.1
Oscar Dollie,Shoe Thieves,25.8,5.9,7.3,6.0,6.6
Tiana Wheeler,Magic,14.8,4.7,2.1,2.4,5.6


### Fringe Benefit
**Recruit a Random Player with the Perk Modification to your Team.**

See below for list of all active players with Perk. This will not remove an existing player from our team. Note that
Perk adds to eDensity.

In [31]:
attractors = []
for team in all_teams:
    attractors.extend(has_mods(team.lineup + team.rotation, "PERK"))
display(get_stars(attractors, include_team=True))

,,Combined,Batting,Pitching,Baserunning,Defense
,Team,,,,,
Baby Doyle,Jazz Hands,17.0,5.5,2.9,3.7,4.8
Glabe Moon,Flowers,16.5,3.6,2.0,4.9,6.0
Parker Meng,Worms,14.2,2.3,5.1,3.0,3.9
Sandie Turner,Millennials,11.3,3.4,3.1,3.1,1.7
Holden Stanton,Steaks,11.0,3.3,0.7,3.0,4.0
Knight Triumphant,Steaks,21.3,6.6,5.0,4.6,5.1
Commissioner Vapor,Moist Talkers,20.5,6.4,4.5,5.5,4.1
Beans McBlase,Moist Talkers,17.0,4.4,4.1,2.8,5.7
Cornelius Games,Shoe Thieves,10.1,1.4,4.0,1.3,3.4


### Pitch Runner
**Move the Best Baserunner in your Team's Rotation to a random spot in your Team's Lineup.**

This would move the following player to our lineup.

In [32]:
temp_pies = pies.rotation.copy()
temp_pies.sort(key=lambda x: x.get_baserunning_rating(), reverse=True)
display(get_stars(temp_pies[0]))
set_heatmap(get_batting_stlats([temp_pies[0]] + pies.lineup), maxVal=1.0)

,Combined,Batting,Pitching,Baserunning,Defense
Tiana Takahashi,16.6,2.7,5.3,4.0,4.6


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Tiana Takahashi,0.542382,0.637924,0.269372,1.254455,1.127316,0.232383,0.666220,0.001000,0.393284
Eduardo Woodman,1.126442,1.345315,1.152985,0.970888,0.876160,0.283687,0.749895,0.001000,1.025183
Beasley Day,1.072655,1.213448,1.089350,0.794653,0.919026,0.037671,0.903595,0.001000,1.165646
Huber Frumple,0.958063,0.606219,1.424455,1.034082,1.290501,0.142676,0.762523,0.001000,1.182151
Ruslan Greatness,0.951096,0.622589,1.145689,1.569998,1.470457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.427757,1.708245,1.164850,1.840876,2.077422,0.001000,2.089850,0.001000,1.530694
Lang Richardson,0.705895,0.884401,0.478324,0.685473,0.990725,0.129280,0.570472,0.001000,0.625391
Peanut Holloway,0.902187,0.813171,1.014025,1.047254,0.996649,0.001000,0.272813,0.716477,1.048080
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176


### Slowpoke
**Move the Worst Baserunner in your Team's Lineup to a random spot in your Team's Rotation.**

This would move the following player to our rotation.

In [33]:
temp_pies = pies.lineup.copy()
temp_pies.sort(key=lambda x: x.get_baserunning_rating())
display(get_stars(temp_pies[0]))
set_heatmap(get_pitching_stlats([temp_pies[0]] + pies.rotation), maxVal=1.0)

,Combined,Batting,Pitching,Baserunning,Defense
Usurper Violet,11.7,2.9,3.2,1.7,3.9


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Usurper Violet,0.635537,0.916861,0.580602,0.980022,0.172289,0.582188,0.847305
Tiana Takahashi,1.064998,1.142441,1.215111,1.182198,0.356126,0.872711,1.199716
Greer Lott,1.233730,1.619256,1.082263,1.033680,0.497049,1.099649,0.471501
Henry Marshallow,0.812533,0.523573,1.209625,1.470785,0.867855,0.856246,0.905731


### Stuck in the Freezer
**A Random Player in your Team's Rotation will have their Coldness Boosted by 40%.**

Coldness is currently unknown, though potentially prevents stolen bases. See below for change in player stars and stlats.

In [34]:
new_team = []
for player in real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"coldness": 0.4}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

,Combined,Batting,Pitching,Baserunning,Defense
Tiana Takahashi,16.6,2.7,5.4,4.0,4.6
Greer Lott,19.0,3.9,6.2,3.9,5.0
Henry Marshallow,15.6,3.8,4.1,2.9,4.8


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Tiana Takahashi,1.075091,1.142441,1.215111,1.182198,0.356126,1.272711,1.199716
Greer Lott,1.243336,1.619256,1.082263,1.033680,0.497049,1.499649,0.471501
Henry Marshallow,0.820357,0.523573,1.209625,1.470785,0.867855,1.256246,0.905731


### Shot Caller
**A Random Player in your Team's Lineup will be 40% better at landing Home Runs in a Big Bucket.**

See below for list of teams that currently have Big Buckets in their Stadium.

In [35]:
stadiums = Stadium.load_all()
bucket_teams = has_mods(stadiums, "BIG_BUCKET")
table = pandas.DataFrame()
for s in bucket_teams:
    table = table.append(pandas.Series({"Stadium Name": s.name}, name=s.team_id.nickname))
table

,Stadium Name
Firefighters,The Fire Safety Awareness Center
Breath Mints,Kansas City FreshDome
Wild Wings,Basin of Mexico Park
Magic,Yellowstone National (Ball)Park
Tacos,Al Pastor Memorial Park
Spies,An Undisclosed Location
Steaks,George Fourman Stadium
Tigers,Sixth Circle Stadium
Dale,Miami International Arena
Pies,Tastycake Stadium


### Cool Off
**All Players in your Team's Rotation will gain 20% Coldness but lose 5% Ruthlessness.**

Coldness is unknown, and Ruthlessness is more correlated to increased strikes and fewer balls thrown. See below for
change in team stats.

In [36]:
new_team = []
for player in real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"coldness": 0.2, "ruthlessness": -0.05}))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Tiana Takahashi,1.052664,1.142441,1.165111,1.182198,0.356126,1.072711,1.199716
Greer Lott,1.215675,1.619256,1.032263,1.033680,0.497049,1.299649,0.471501
Henry Marshallow,0.803132,0.523573,1.159625,1.470785,0.867855,1.056246,0.905731


### Global Guts
**All Players in your Team's Lineup will gain 20% Moxie but lose 5% Thwackability.**

Moxie correlates to better plate discipline (more walks), and thwackability correlates to better contact. See below for
change in team stats.

In [37]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"moxie": 0.2, "thwackability": -0.05}))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.128883,1.295315,1.152985,0.970888,1.076160,0.283687,0.749895,0.001000,1.025183
Beasley Day,1.072698,1.163448,1.089350,0.794653,1.119026,0.037671,0.903595,0.001000,1.165646
Huber Frumple,0.939729,0.556219,1.424455,1.034082,1.490501,0.142676,0.762523,0.001000,1.182151
Ruslan Greatness,0.932509,0.572589,1.145689,1.569998,1.670457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.422764,1.658245,1.164850,1.840876,2.277422,0.001000,2.089850,0.001000,1.530694
Lang Richardson,0.701267,0.834401,0.478324,0.685473,1.190725,0.129280,0.570472,0.001000,0.625391
Peanut Holloway,0.894556,0.763171,1.014025,1.047254,1.196649,0.001000,0.272813,0.716477,1.048080
Usurper Violet,0.577051,0.844648,0.293503,1.022726,1.069795,0.569699,0.283561,0.100000,0.487176


### Backup or Down
**Flip the eDensity of the most eDense Player in your Team's Shadows.**

This would make the most dense shadows player have the Negative modification, making their eDensity negative.

In [38]:
temp_pies = pies.shadows.copy()
temp_pies.sort(key=lambda x: x.e_density, reverse=True)
display(f"{temp_pies[0].name} eDensity change: {temp_pies[0].e_density} to {-temp_pies[0].e_density}")
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*temp_pies[0].e_density}")

'Doc Anice eDensity change: 39.81715027609057 to -39.81715027609057'

'Total Team eDensity change: 2074.8437229733454 to 1995.2094224211642'

### Complete 180

**Flip the eDensity of the most eDense Player in your Team's Rotation.**

This would make the most dense pitcher have the Negative modification, making their eDensity negative.

In [39]:
temp_pies = pies.rotation.copy()
temp_pies.sort(key=lambda x: x.e_density, reverse=True)
display(f"{temp_pies[0].name} eDensity change: {temp_pies[0].e_density} to {-temp_pies[0].e_density}")
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*temp_pies[0].e_density}")

'Henry Marshallow eDensity change: 31.752575266463545 to -31.752575266463545'

'Total Team eDensity change: 2074.8437229733454 to 2011.3385724404184'

### Head over Tails

**Flip the eDensity of the most eDense Player in your Team's Lineup.**

This would make the most dense batter have the Negative modification, making their eDensity negative.

In [40]:
temp_pies = pies.lineup.copy()
temp_pies.sort(key=lambda x: x.e_density, reverse=True)
display(f"{temp_pies[0].name} eDensity change: {temp_pies[0].e_density} to {-temp_pies[0].e_density}")
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*temp_pies[0].e_density}")

'Jaxon Buckley eDensity change: 47.686846067178365 to -47.686846067178365'

'Total Team eDensity change: 2074.8437229733454 to 1979.4700308389886'

### Low Five

**Flip the eDensity of the five least eDense Players on your Team's Roster.**

See below for list of players and their eDensity impact.

In [41]:
temp_pies = pies.lineup.copy() + pies.rotation.copy() + pies.shadows.copy()
temp_pies.sort(key=lambda x: x.e_density)
table = pandas.DataFrame()
for p in temp_pies[0:5]:
    table = table.append(pandas.Series({"eDensity": p.e_density}, name=p.name))
display(table)
density_sum = sum([x.e_density for x in temp_pies[0:5]])
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*density_sum}")

,eDensity
Greer Lott,-36.560422
Peanut Holloway,-22.987620
Nerd Pacheco,10.323060
Bambi Perez,14.032362
Mummy Cantburn,16.639602


'Total Team eDensity change: 2074.8437229733454 to 2111.949759214704'

### Slow Clap

**The first Player in your Team's Lineup will gain the Slow Build mod.**

Slow Build makes players bat 1% better for each At Bat in a game, resetting at the beginning of each game.

### States of Play

**Give all Elsewhere or Scattered Players on your Team's Roster the Undefined mod.**

Undefined makes a player play 50-100% better while Scattered. See below for list of Elsewhere of Scattered players.

In [42]:
scattered = has_mods(pies.lineup + pies.rotation + pies.shadows, ["SCATTERED", "ELSEWHERE"])
get_stars(scattered)

""


### Solo Stroll

**The worst Batter in your Team's Lineup will draw a walk on 3 balls instead of 4.**

This would add permanent Walk in the Park to our worst batter, listed below.

In [43]:
get_stars(worst_batter)

,Combined,Batting,Pitching,Baserunning,Defense
Usurper Violet,11.7,2.9,3.2,1.7,3.9


### Targeted Evolution

**A random pitcher and a random hitter on your Team will Advance.**

An Advanced player will gain one evolution level, increasing their minimum star level by one for all stats and raising
their stars to the minimum if they are lower than that. It also allows players to hold an additional item.

## Gifts

Gifts are awarded to teams based on contributions from other teams. Each team creates a Wishlist of items that they
would like, and the top items on the wishlist are awarded during the Latesiesta.

### eDense Infusion

**Ratings Boost! Add the eDense Element to 20% of the non-eDense Items held by Players on your Team.**

It is unknown what item eDensity is or how it effects the sim.

### Collector's Editions

Collectors Editions move a Legendary player to the team. Replicas act as copies of that player and will turn to Dust
after elections.

Special Notes:
* Nagomi has fake Attractor stars, this analysis uses the player attributes.
* Chorby is still unstable, and is likely to get incinerated if possible.
* Chorby replicas have 0 soul, so are safe from consumers

In [44]:
york = Player.find_by_name("York Silk")
nagomi = Player.find_by_name("Nagomi Mcdaniel")
nagomi.batting_rating = None
nagomi.baserunning_rating = None
nagomi.defense_rating = None
aldon = Player.find_by_name("Aldon Cashmoney")
goodwin = Player.find_by_name("Goodwin Morin")
val = Player.find_by_name("Valentine Games")
pm = Player.find_by_name("Pitching Machine")
chorby = Player.find_by_name("Chorby Soul")

display(Markdown("#### Batting"))
display(set_heatmap(get_batting_stlats([chorby, val, aldon, goodwin, pm] + pies.lineup), maxVal=1.5))

display(Markdown("Adding Chorby Soul"))
display(pandas.DataFrame(move_player(pies, chorby, position='lineup')))

display(Markdown("Adding Valentine Games"))
display(pandas.DataFrame(move_player(pies, val, position='lineup')))

display(Markdown("Adding Aldon Cashmoney"))
display(pandas.DataFrame(move_player(pies, aldon, position='lineup')))

display(Markdown("Adding Goodwin Morin"))
display(pandas.DataFrame(move_player(pies, goodwin, position='lineup')))

display(Markdown("Adding Pitching Machine"))
display(pandas.DataFrame(move_player(pies, pm, position='lineup')))

display(Markdown("#### Pitching"))
display(set_heatmap(get_pitching_stlats([nagomi, chorby] + pies.rotation), maxVal=1.5))

display(Markdown("Adding Nagomi McDaniel"))
display(pandas.DataFrame(move_player(pies, nagomi, position='rotation')))

display(Markdown("Adding Chorby Soul"))
display(pandas.DataFrame(move_player(pies, chorby, position='rotation')))

#### Batting

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Chorby Soul,0.001820,0.001000,0.001000,0.001000,0.001000,0.990000,0.001000,0.999000,0.001000
Valentine Games,1.322240,1.237714,1.407875,1.829484,1.477367,0.001000,1.685680,0.001000,1.274472
Aldon Cashmoney,0.998344,1.058616,1.070545,0.723468,1.110157,0.342420,0.728757,0.158430,0.415462
Goodwin Morin,1.116151,1.011205,1.054999,1.804185,1.724130,0.001000,1.146978,0.054253,1.040647
Pitching Machine,0.968126,0.637427,1.256693,1.400732,1.401861,0.154714,1.164864,0.001000,1.399386
Eduardo Woodman,1.126442,1.345315,1.152985,0.970888,0.876160,0.283687,0.749895,0.001000,1.025183
Beasley Day,1.072655,1.213448,1.089350,0.794653,0.919026,0.037671,0.903595,0.001000,1.165646
Huber Frumple,0.958063,0.606219,1.424455,1.034082,1.290501,0.142676,0.762523,0.001000,1.182151
Ruslan Greatness,0.951096,0.622589,1.145689,1.569998,1.470457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.427757,1.708245,1.164850,1.840876,2.077422,0.001000,2.089850,0.001000,1.530694


Adding Chorby Soul

,Pies
batting_change,-10.7073%
pitching_change,+0.0000%
baserunning_change,-8.1979%
defense_change,-8.4916%


Adding Valentine Games

...

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Nagomi Mcdaniel,0.862040,0.917836,0.608354,0.703359,1.145577,0.994793,0.560973
Chorby Soul,0.000299,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000
Tiana Takahashi,1.064998,1.142441,1.215111,1.182198,0.356126,0.872711,1.199716
Greer Lott,1.233730,1.619256,1.082263,1.033680,0.497049,1.099649,0.471501
Henry Marshallow,0.812533,0.523573,1.209625,1.470785,0.867855,0.856246,0.905731


Adding Nagomi McDaniel

,Pies
batting_change,+0.0000%
pitching_change,-4.3762%
baserunning_change,+0.0000%
defense_change,+0.0000%


Adding Chorby Soul

,Pies
batting_change,+0.0000%
pitching_change,-25.9197%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Solo Editions

These gifts provide a random player with a benefit for the lateseason and postseason.

* Ambitious: Player will overperform in the postseason.
* Unambitious: Unknown, presumably the opposite of Ambitious.

### Team Editions

These gifts provide a team-wide benefit for the lateseason and postseason.

* Late to the Party: Team will overperform in the Lateseason.
* Early to the Party: Unknown, presumably the opposite of Late to the Party.
* Fireproof: Team will be safe from incinerations. Pies only have 1 Solar Eclipse game in the lateseason.
* Soundproof: Team will be safe from Feedback. Pies have 2 Feedback weather games in the lateseason.
* Gravity: Team will be safe from Reverb. Pies only have 1 Reverb weather game in the lateseason.

### Handcrafted Drops

These will provide Items which have additional stat increases and random special effects.

|Type | Attribute | Stat |
| --- | --- | --- |
| Bat | +Thwackability | Batting |
| Cap | +Unthwackability | Pitching |
| Phone | ??? | ??? |
| Quill | ??? | ??? |

See below for a list of players that can receive them.

The Bats will have Subtractor, converting Runs to UnRuns and vice-versa. The Caps will have Underhanded which will
convert earned Home Runs to UnRuns.

**Bat**

In [45]:
free_items_lineup = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_lineup)

,Combined,Batting,Pitching,Baserunning,Defense
Peanut Holloway,12.1,4.5,1.5,2.4,3.7


**Cap, Quill**

In [46]:
free_items_rotation= [x for x in pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_rotation)

""


**Phone**

In [47]:
free_items = [x for x in pies.lineup + pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items)

,Combined,Batting,Pitching,Baserunning,Defense
Peanut Holloway,12.1,4.5,1.5,2.4,3.7


### Soul Patches

**Back by Popular Demand! Give 1 Soul to your least Soul Full player. Repeat 10 times.**

See below for final Soul totals. Low soul is dangerous, as Players will become Redacted if their soul hits 0 and
Consumers will drain 1 soul per chomp.

In [48]:
players = deepcopy(pies.rotation) + deepcopy(pies.lineup)
for i in range(0, 10):
    players.sort(key=lambda x: x.soul)
    worst = players[0]
    worst.soul += 1
    players = [x for x in players if x.id != worst.id] + [worst]
players.sort(key=lambda x: x.soul)
x = pandas.DataFrame()
for p in players:
    old = [x for x in pies.rotation + pies.lineup if x.id == p.id][0]
    x = x.append(pandas.Series({"Soul Added": p.soul - old.soul,"Resulting Soul": p.soul}, name=p.name))
x

,Resulting Soul,Soul Added
Ruslan Greatness,5.0,1.0
Usurper Violet,5.0,1.0
Peanut Holloway,5.0,2.0
Lang Richardson,6.0,0.0
Greer Lott,6.0,1.0
Henry Marshallow,6.0,1.0
Tiana Takahashi,6.0,2.0
Eduardo Woodman,6.0,2.0
Beasley Day,7.0,0.0
Huber Frumple,7.0,0.0


### Bargain Bin

**Give a random Player on your Team the Best Item from the Bargain Bin!**

The Bargain Bin will retrieve one random previously dropped item. Below is a list of all non-base items not currently
held by players.

In [49]:
all_items = Item.load_all()
rare_items = [x for x in all_items.values() if len(x.adjustments) > 1]
carried_items = get_held_items()
dropped_items = [x for x in rare_items if x not in carried_items]
dropped_items.sort(key=lambda x: len(x.adjustments), reverse=True)

get_item_stats(dropped_items)

,Max Durability,Modifications
Weird Plant-Based Bat,3,[]
Hearty Rubber Cap,3,[]
Slimy Ring of Vitality,3,[]
Sharp Jersey of Charisma,3,[]
Hot Ring of Intelligence,3,[]
Aluminum Blood Sunglasses,3,[]
Sharp Leg Bat,3,[]
Holey Sunglasses of Strength,3,[]
Rubber Skate Cannon,3,[Pro Skater]
Weird Shoes,2,[]


---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
